In [1]:
import json
import sys,os
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

sys.path.extend(['/root/xiaoda/query_topic/'])

In [3]:
import torch
from torch.nn import functional as F
import numpy as np
import random
import torch.nn as nn
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

"""
https://github.com/ondrejbohdal/meta-calibration/blob/main/Metrics/metrics.py
"""

class ECE(nn.Module):
    
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(ECE, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels, mode='logits'):
        if mode == 'logits':
            softmaxes = F.softmax(logits, dim=1)
        else:
            softmaxes = logits
        # softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)
        
        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece

In [34]:
import torch
import json
import sys
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizerFast
import transformers
from datetime import timedelta

import os, sys

from nets.them_classifier import MyBaseModel, RobertaClassifier

import configparser
from tqdm import tqdm

cur_dir_path = '/root/xiaoda/query_topic/'

def load_label(filepath):
    label_list = []
    with open(filepath, 'r') as frobj:
        for line in frobj:
            label_list.append(line.strip())
        n_classes = len(label_list)

        label2id = {}
        id2label = {}
        for idx, label in enumerate(label_list):
            label2id[label] = idx
            id2label[idx] = label
        return label2id, id2label

class RiskInfer(object):
    def __init__(self, config_path):

        import torch, os, sys

        con = configparser.ConfigParser()
        con_path = os.path.join(cur_dir_path, config_path)
        con.read(con_path, encoding='utf8')

        args_path = dict(dict(con.items('paths')), **dict(con.items("para")))
        self.tokenizer = BertTokenizerFast.from_pretrained(args_path["model_path"], do_lower_case=True)

        from collections import OrderedDict
        self.schema_dict = OrderedDict({})

        for label_index, schema_info in enumerate(args_path["label_path"].split(',')):
            schema_type, schema_path = schema_info.split(':')
            schema_path = os.path.join(cur_dir_path, schema_path)
            print(schema_type, schema_path, '===schema-path===')
            label2id, id2label = load_label(schema_path)
            self.schema_dict[schema_type] = {
                'label2id':label2id,
                'id2label':id2label,
                'label_index':label_index
            }
            print(self.schema_dict[schema_type], '==schema_type==', schema_type)
        
        output_path = os.path.join(cur_dir_path, args_path['output_path'])

        from roformer import RoFormerModel, RoFormerConfig

        config = RoFormerConfig.from_pretrained(args_path["model_path"])
        encoder = RoFormerModel(config=config)
        
        encoder_net = MyBaseModel(encoder, config)

        self.device = "cuda:3" if torch.cuda.is_available() else "cpu"

        classifier_list = []

        schema_list = list(self.schema_dict.keys())

        for schema_key in schema_list:
            classifier = RobertaClassifier(
                hidden_size=config.hidden_size, 
                dropout_prob=con.getfloat('para', 'out_dropout_rate'),
                num_labels=len(self.schema_dict[schema_key]['label2id']), 
                dropout_type=con.get('para', 'dropout_type'))
            classifier_list.append(classifier)

        classifier_list = nn.ModuleList(classifier_list)

        class MultitaskClassifier(nn.Module):
            def __init__(self, transformer, classifier_list):
                super().__init__()

                self.transformer = transformer
                self.classifier_list = classifier_list

            def forward(self, input_ids, input_mask, 
                        segment_ids=None, 
                        transformer_mode='mean_pooling', 
                        dt_idx=None):
                hidden_states = self.transformer(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              return_mode=transformer_mode)
                outputs_list = []
                
                for idx, classifier in enumerate(self.classifier_list):
                    
                    if dt_idx is not None and idx != dt_idx:
                        continue
                    
                    ce_logits = classifier(hidden_states)
                    outputs_list.append(ce_logits)
                return outputs_list, hidden_states

        self.net = MultitaskClassifier(encoder_net, classifier_list).to(self.device)

        # eo = 9
        # ckpt = torch.load(os.path.join(output_path, 'multitask_cls.pth.{}.raw'.format(eo)), map_location=self.device)
        # # ckpt = torch.load(os.path.join(output_path, 'multitask_cls.pth.{}.raw.focal'.format(eo)), map_location=self.device)
        # # ckpt = torch.load(os.path.join(output_path, 'multitask_contrast_cls.pth.{}'.format(eo)), map_location=self.device)
        # self.net.load_state_dict(ckpt)
        # self.net.eval()
        
    def reload(self, model_path):
        ckpt = torch.load(model_path, map_location=self.device)
        self.net.load_state_dict(ckpt)
        self.net.eval()

    def predict(self, text):

        """抽取输入text所包含的类型
        """
        encoder_txt = self.tokenizer.encode_plus(text, max_length=256)
        input_ids = torch.tensor(encoder_txt["input_ids"]).long().unsqueeze(0).to(self.device)
        token_type_ids = torch.tensor(encoder_txt["token_type_ids"]).unsqueeze(0).to(self.device)
        attention_mask = torch.tensor(encoder_txt["attention_mask"]).unsqueeze(0).to(self.device)
        
        scores_dict = {}
        with torch.no_grad():
            [logits_list, 
            hidden_states] = self.net(input_ids, 
                attention_mask, token_type_ids, transformer_mode='cls')
        for schema_type, logits in zip(list(self.schema_dict.keys()), logits_list):
            scores = torch.nn.Softmax(dim=1)(logits)[0].data.cpu().numpy()
            scores_dict[schema_type] = []
            for index, score in enumerate(scores):
                scores_dict[schema_type].append([self.schema_dict[schema_type]['id2label'][index], 
                                        float(score)])
        return scores_dict
    
    def get_logitnorm(self, text):
        """抽取输入text所包含的类型
        """
        encoder_txt = self.tokenizer.encode_plus(text, max_length=256)
        input_ids = torch.tensor(encoder_txt["input_ids"]).long().unsqueeze(0).to(self.device)
        token_type_ids = torch.tensor(encoder_txt["token_type_ids"]).unsqueeze(0).to(self.device)
        attention_mask = torch.tensor(encoder_txt["attention_mask"]).unsqueeze(0).to(self.device)
        
        scores_dict = {}
        logits_norm_list = []
        with torch.no_grad():
            [logits_list, 
            hidden_states] = self.net(input_ids, 
                attention_mask, token_type_ids, transformer_mode='cls')
            for logits in logits_list:
                logits_norm_list.append(logits/torch.norm(logits, p=2, dim=-1, keepdim=True) + 1e-7)
        for schema_type, logit_norm in zip(list(self.schema_dict.keys()), logits_norm_list):
            scores_dict[schema_type] = logit_norm[0].data.cpu().numpy()
        return scores_dict
            
    
    def predict_batch(self, text):
        if isinstance(text, list):
            text_list = text
        else:
            text_list = [text]
        model_input = self.tokenizer(text_list, return_tensors="pt",padding=True)
        for key in model_input:
            model_input[key] = model_input[key].to(self.device)
        with torch.no_grad():
            [logits_list, 
            hidden_states] = self.net(model_input['input_ids'], 
                model_input['attention_mask'], 
                model_input['token_type_ids'], transformer_mode='cls')
        score_dict_list = []
        for idx, text in enumerate(text_list):
            scores_dict = {}
            for schema_type, logits in zip(list(self.schema_dict.keys()), logits_list):
                scores = torch.nn.Softmax(dim=1)(logits)[idx].data.cpu().numpy()
                scores_dict[schema_type] = []
                for index, score in enumerate(scores):
                    scores_dict[schema_type].append([self.schema_dict[schema_type]['id2label'][index], 
                                            float(score)])
            score_dict_list.append(scores_dict)
        return score_dict_list

# risk_api = RiskInfer('./risk_data/config.ini')
risk_api = RiskInfer('./risk_data_v5/config_offensive_risk_senti.ini')




offensive /data/albert.xht/xiaoda/sentiment/offensive/offensive_label.txt ===schema-path===
{'label2id': {'冒犯': 0, '正常': 1}, 'id2label': {0: '冒犯', 1: '正常'}, 'label_index': 0} ==schema_type== offensive
query_risk /data/albert.xht/xiaoda/sentiment/query_risk_v11/query_risk_label.txt ===schema-path===
{'label2id': {'风险': 0, '个人信息': 1, '正常': 2}, 'id2label': {0: '风险', 1: '个人信息', 2: '正常'}, 'label_index': 1} ==schema_type== query_risk
bias /data/albert.xht/xiaoda/sentiment/bias/bias_label.txt ===schema-path===
{'label2id': {'偏见': 0, '正常': 1}, 'id2label': {0: '偏见', 1: '正常'}, 'label_index': 2} ==schema_type== bias
ciron /data/albert.xht/xiaoda/sentiment/ciron/ciron_label.txt ===schema-path===
{'label2id': {'讽刺': 0, '正常': 1}, 'id2label': {0: '讽刺', 1: '正常'}, 'label_index': 3} ==schema_type== ciron
senti /data/albert.xht/xiaoda/sentiment/senti/senti_label.txt ===schema-path===
{'label2id': {'负向': 0, '正向': 1}, 'id2label': {0: '负向', 1: '正向'}, 'label_index': 4} ==schema_type== senti


01/06/2023 15:10:27 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/06/2023 15:10:27 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/06/2023 15:10:27 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/06/2023 15:10:27 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/06/2023 15:10:27 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++


In [40]:
risk_api.reload('/data/albert.xht/xiaoda/risk_classification/multitask_raw_filter_senti_query_risk_v11_offensive_v4/multitask_cls.pth.9')


In [37]:
from sklearn.metrics import classification_report
from tqdm import tqdm
import re

def eval_all(data, model, key):
    pred = []
    gold = []
    pred_score = []
    for item in tqdm(data):
        gold.append(item['label'][0])
        if isinstance(item['text'], list):
            text = "\n".join(item['text'])
        else:
            text = item['text']
        text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", "", text)   # 合并正文中过多的空格

        result = model.predict(text)
        score = sorted(result[key], key=lambda u:u[1], reverse=True)
        pred.append(score[0][0])
        pred_score.append(result[key])
    print(classification_report(gold, pred, digits=4))
    return pred, gold, pred_score
    
def evaluation_ece(pred_score, gold):
    pred_score_l = []
    mapping_dict = {}
    for item in pred_score:
        pred_score_l.append([])
        for idx, p in enumerate(item):
            if p[0] not in mapping_dict:
                mapping_dict[p[0]] = idx
            pred_score_l[-1].append(p[1])
    pred_score_l = torch.tensor(pred_score_l)
    gold_l = torch.tensor([mapping_dict[item] for item in gold])

    ece_fn = ECE(n_bins=15)
    print(ece_fn(pred_score_l, gold_l, mode='probs'), '==ece==')
# pred, gold, pred_score = eval_all(offensive_test, risk_api, 'offensive')
# evaluation_ece(pred_score, gold)


In [6]:

risk_api.reload('/data/albert.xht/xiaoda/risk_classification/multitask_raw_filter_senti_query_risk_v11_offensive_v3/multitask_cls.pth.9')

# risk_query = []
# with open('/data/albert.xht/xiaodao/query_risk_v11/offensive_select_labeled.txt') as frobj:
#     for line in frobj:
#         risk_query.append(json.loads(line.strip()))
# pred, gold, pred_score = eval_all(risk_query, risk_api, 'query_risk')
# evaluation_ece(pred_score, gold)


In [41]:
offensive = []
with open('/data/albert.xht/sentiment/test/offensive_cold.json') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        offensive.append(content)
        
pred, gold, pred_score = eval_all(offensive, risk_api, 'offensive')
evaluation_ece(pred_score, gold)

100%|██████████| 5304/5304 [00:44<00:00, 119.47it/s]


              precision    recall  f1-score   support

          冒犯     0.7261    0.8609    0.7877      2106
          正常     0.8956    0.7861    0.8373      3198

    accuracy                         0.8158      5304
   macro avg     0.8108    0.8235    0.8125      5304
weighted avg     0.8283    0.8158    0.8176      5304

tensor([0.1220]) ==ece==


In [173]:

risk_api.reload('/data/albert.xht/xiaoda/risk_classification/multitask_raw_filter_senti_query_risk_v11_offensive_v2/multitask_cls.pth.9')

risk_query = []
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/offensive_select_labeled.txt.paraphrase') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        for d in content['paraphrase']:
            p = {
                'text':d[0],
                'label':content['label']
            }
            if d:
                risk_query.append(p)
        # risk_query.append(json.loads(line.strip()))
        
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/offensive_select_labeled.evaluation', 'w') as fwobj:
    for d in risk_query:
        fwobj.write(json.dumps(d, ensure_ascii=False)+'\n')
        
pred, gold, pred_score = eval_all(risk_query, risk_api, 'query_risk')
evaluation_ece(pred_score, gold)


100%|██████████| 83982/83982 [10:44<00:00, 130.31it/s]
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        个人信息     0.0000    0.0000    0.0000         0
          正常     0.7531    0.7944    0.7732     23186
          风险     0.9199    0.9007    0.9102     60796

    accuracy                         0.8713     83982
   macro avg     0.5577    0.5650    0.5611     83982
weighted avg     0.8739    0.8713    0.8724     83982

tensor([0.0406]) ==ece==


In [182]:
t = []
for l, p in zip(pred, risk_query):
    if l == '个人信息':
        t.append(p)


[{'text': '"要饭的不嫌饭馊"?日前家里打扫卫生.沙发陈旧,想送给破烂王.谁"要饭的不嫌饭馊"?日前家里打扫卫生.沙发陈旧,想送给破烂王.谁能用过的,我会看不到,不会收拾东西',
  'label': ['风险']}]

In [45]:
risk_api.predict('睾丸')

{'offensive': [['冒犯', 0.46404173970222473], ['正常', 0.5359582901000977]],
 'query_risk': [['风险', 0.3581663966178894],
  ['个人信息', 5.1983366574859247e-05],
  ['正常', 0.6417816281318665]],
 'bias': [['偏见', 0.838456392288208], ['正常', 0.161543607711792]],
 'ciron': [['讽刺', 1.2640751265280414e-05], ['正常', 0.9999873638153076]],
 'senti': [['负向', 0.221103236079216], ['正向', 0.7788968086242676]]}

In [42]:
risk_query = []
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/offensive_select_labeled.txt.paraphrase') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        # for d in content['paraphrase']:
        #     p = {
        #         'text':d[0],
        #         'label':content['label']
        #     }
        #     if d:
        #         risk_query.append(p)
        risk_query.append(json.loads(line.strip()))
pred, gold, pred_score = eval_all(risk_query, risk_api, 'query_risk')
evaluation_ece(pred_score, gold)

100%|██████████| 20641/20641 [02:51<00:00, 120.61it/s]


              precision    recall  f1-score   support

          正常     0.8761    0.7833    0.8271      5514
          风险     0.9239    0.9596    0.9414     15127

    accuracy                         0.9125     20641
   macro avg     0.9000    0.8714    0.8843     20641
weighted avg     0.9111    0.9125    0.9109     20641

tensor([0.0069]) ==ece==


In [5]:
from tqdm import tqdm
import numpy as np
import json, re

def risk_predict_batch(risk_api, text):
    if isinstance(text, list):
        text_list = text
    else:
        text_list = [text]
    result_list = risk_api.predict_batch(text_list)
    return result_list

In [20]:
offensive = []
with open('/data/albert.xht/xiaodao/topic_classification_v7/biake_qa_web_text_zh_train.json.positive', 'r') as frobj:
    queue = []
    t = []
    for line in tqdm(frobj):
        content = json.loads(line.strip())
        content['text'] = re.sub('如何评价', '', content['text'])
        text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", "", content['text'])   # 合并正文中过多的空格
        queue.append(text)
        t.append(content)
        if np.mod(len(queue), 512) == 0 and queue:
            probs = risk_predict_batch(risk_api, queue)
            for prob_dict, text, tt in zip(probs, queue, t):
                content = {
                    'text':text,
                    'topic':tt['topic'],
                    'score_list':prob_dict
                }
                offensive.append(content)
            queue = []
            t = []
    if queue:
        probs = risk_predict_batch(risk_api, queue)
        for prob_dict, text, tt in zip(probs, queue, t):
            content = {
                'text':text,
                'topic':tt['topic'],
                'score_list':prob_dict
            }
            offensive.append(content)

25507it [00:10, 2540.99it/s]


In [6]:
# coding=utf-8
import os
import re
import json
import numpy as np
import pandas as pd
import html
import urllib
import jieba
import jieba.posseg as pseg
import w3lib.html
import logging
import warnings
from tqdm import tqdm
from pypinyin import lazy_pinyin, pinyin
from opencc import OpenCC
from collections import defaultdict


def harvest_clean_text(text, remove_url=True, email=True, weibo_at=True, stop_terms=("转发微博",),
                       emoji=True, weibo_topic=False, deduplicate_space=True,
                       norm_url=False, norm_html=False, to_url=False,
                       remove_puncts=False, remove_tags=True, t2s=False,
                       expression_len=(1, 6), linesep2space=False):
    '''
    进行各种文本清洗操作，微博中的特殊格式，网址，email，html代码，等等
    :param text: 输入文本
    :param remove_url: （默认使用）是否去除网址
    :param email: （默认使用）是否去除email
    :param weibo_at: （默认使用）是否去除微博的\@相关文本
    :param stop_terms: 去除文本中的一些特定词语，默认参数为("转发微博",)
    :param emoji: （默认使用）去除\[\]包围的文本，一般是表情符号
    :param weibo_topic: （默认不使用）去除##包围的文本，一般是微博话题
    :param deduplicate_space: （默认使用）合并文本中间的多个空格为一个
    :param norm_url: （默认不使用）还原URL中的特殊字符为普通格式，如(%20转为空格)
    :param norm_html: （默认不使用）还原HTML中的特殊字符为普通格式，如(\&nbsp;转为空格)
    :param to_url: （默认不使用）将普通格式的字符转为还原URL中的特殊字符，用于请求，如(空格转为%20)
    :param remove_puncts: （默认不使用）移除所有标点符号
    :param remove_tags: （默认使用）移除所有html块
    :param t2s: （默认不使用）繁体字转中文
    :param expression_len: 假设表情的表情长度范围，不在范围内的文本认为不是表情，不加以清洗，如[加上特别番外荞麦花开时共五册]。设置为None则没有限制
    :param linesep2space: （默认不使用）把换行符转换成空格
    :return: 清洗后的文本
    '''
    # unicode不可见字符
    # 未转义
    text = re.sub(r"[\u200b-\u200d]", "", text)
    # 已转义
    text = re.sub(r"(\\u200b|\\u200c|\\u200d)", "", text)
    # 反向的矛盾设置
    if norm_url and to_url:
        raise Exception("norm_url和to_url是矛盾的设置")
    if norm_html:
        text = html.unescape(text)
    if to_url:
        text = urllib.parse.quote(text)
    if remove_tags:
        text = w3lib.html.remove_tags(text)
    if remove_url:
        try:
            URL_REGEX = re.compile(
                r'(?i)http[s]?://(?:[a-zA-Z]|[0-9]|[#$%*-;=?&@~.&+]|[!*,])+',
                re.IGNORECASE)
            text = re.sub(URL_REGEX, "", text)
        except:
            # sometimes lead to "catastrophic backtracking"
            zh_puncts1 = "，；、。！？（）《》【】"
            URL_REGEX = re.compile(
                r'(?i)((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>' + zh_puncts1 +
                ']+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’' + zh_puncts1 + ']))',
                re.IGNORECASE)
            text = re.sub(URL_REGEX, "", text)
    if norm_url:
        text = urllib.parse.unquote(text)
    if email:
        EMAIL_REGEX = re.compile(
            r"[-a-z0-9_.]+@(?:[-a-z0-9]+\.)+[a-z]{2,6}", re.IGNORECASE)
        text = re.sub(EMAIL_REGEX, "", text)
    if weibo_at:
        text = re.sub(r"(回复)?(//)?(\\\\)?\s*@\S*?\s*(:|：| |$)",
                      " ", text)  # 去除正文中的@和回复/转发中的用户名
    if emoji:
        # 去除括号包围的表情符号
        # ? lazy match避免把两个表情中间的部分去除掉
        if type(expression_len) in {tuple, list} and len(expression_len) == 2:
            # 设置长度范围避免误伤人用的中括号内容，如[加上特别番外荞麦花开时共五册]
            lb, rb = expression_len
            text = re.sub(r"\[\S{"+str(lb)+r","+str(rb)+r"}?\]", "", text)
        else:
            text = re.sub(r"\[\S+?\]", "", text)
        # text = re.sub(r"\[\S+\]", "", text)
        # 去除真,图标式emoji
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)
    if weibo_topic:
        text = re.sub(r"#\S+#", "", text)  # 去除话题内容
    if linesep2space:
        text = text.replace("\n", " ")   # 不需要换行的时候变成1行
    if deduplicate_space:
        text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", r"\1", text)   # 合并正文中过多的空格
        text = re.sub(r"(\s)+", r"\1", text)   # 合并正文中过多的空格
        # text = re.sub(r"(\t)+", r"\1", text)   # 合并正文中过多的空格
    if t2s:
        cc = OpenCC('t2s')
        text = cc.convert(text)
    assert hasattr(stop_terms, "__iter__"), Exception("去除的词语必须是一个可迭代对象")
    if type(stop_terms) == str:
        text = text.replace(stop_terms, "")
    else:
        for x in stop_terms:
            text = text.replace(x, "")
    if remove_puncts:
        allpuncs = re.compile(
            r"[，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+]")
        text = re.sub(allpuncs, "", text)

    return text.strip()

def clean_data(text):
    text = harvest_clean_text(text, remove_url=True, email=True, weibo_at=False, stop_terms=("转发微博",),
                       emoji=True, weibo_topic=True, deduplicate_space=True,
                       norm_url=False, norm_html=False, to_url=False,
                       remove_puncts=False, remove_tags=False, t2s=False,
                       expression_len=(1, 6), linesep2space=False)
    return text


In [10]:

from collections import namedtuple
_DocSpan = namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])

def slide_window(all_doc_tokens, max_length, doc_stride, offset=32):
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_length - offset:
            length = max_length - offset
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
            break
        start_offset += min(length, doc_stride)
    return doc_spans

offensive = []
risk_api.reload('/data/albert.xht/xiaoda/risk_classification/multitask_raw_filter_senti_query_risk_v11_offensive_v2/multitask_cls.pth.9')

with open('/data/albert.xht/xiaoda/sentiment/green_politics/short_text_benchmark_for_cro_albert_politics.json.txt', 'r') as frobj:
    queue = []
    t = []
    for idx, line in tqdm(enumerate(frobj)):
        if idx == 0:
            continue
        content = json.loads(line.strip())
        text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", "", content['text'])   # 合并正文中过多的空格
        # if content['label'] not in ['black']:
        #     continue
        # if len(text) >= 164:
        #     text = text[:164]
        if content['label'] in ['white']:
            spans = slide_window(text[91:], 91, 91)
            for span in spans:
                span_text = text[:91]+text[span.start+91:span.start+span.length+91]
                queue.append(span_text)
                t.append(content)
        elif content['label'] in ['black']:
            queue.append(text[:192])
            t.append(content)
        else:
            continue
        if np.mod(len(queue), 64) == 0 and queue:
            probs = risk_predict_batch(risk_api, queue)
            for prob_dict, text, tt in zip(probs, queue, t):
                content = {
                    'text':text,
                    'topic':tt['label'],
                    'score_list':prob_dict
                }
                offensive.append(content)
            queue = []
            t = []
    if queue:
        probs = risk_predict_batch(risk_api, queue)
        for prob_dict, text, tt in zip(probs, queue, t):
            content = {
                'text':text,
                'topic':tt['label'],
                'score_list':prob_dict
            }
            offensive.append(content)

478540it [07:46, 1026.71it/s]


In [29]:
from collections import namedtuple
_DocSpan = namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])

def slide_window(all_doc_tokens, max_length, doc_stride, offset=32):
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_length - offset:
            length = max_length - offset
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
            break
        start_offset += min(length, doc_stride)
    return doc_spans


    
    


In [15]:
porn = []
risk_api.reload('/data/albert.xht/xiaoda/risk_classification/multitask_raw_filter_senti_query_risk_v11_offensive_v2/multitask_cls.pth.9')

with open('/data/albert.xht/xiaoda/sentiment/green_porn/short_text_benchmark_for_cro_albert_seqing.json.txt', 'r') as frobj:
    queue = []
    t = []
    from collections import Counter
    pppp = Counter()
    for idx, line in tqdm(enumerate(frobj)):
        if idx == 0:
            continue
        content = json.loads(line.strip())
        text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", "", content['text'])   # 合并正文中过多的空格
        # if content['label'] not in ['black']:
        #     continue
        if len(text) >= 164:
            text = text[:164]
        pppp[content['label']] += 1
        queue.append(text)
        t.append(content)
        if np.mod(len(queue), 128) == 0 and queue:
            probs = risk_predict_batch(risk_api, queue)
            for prob_dict, text, tt in zip(probs, queue, t):
                content = {
                    'text':text,
                    'topic':tt['label'],
                    'score_list':prob_dict
                }
                porn.append(content)
            queue = []
            t = []
    if queue:
        probs = risk_predict_batch(risk_api, queue)
        for prob_dict, text, tt in zip(probs, queue, t):
            content = {
                'text':text,
                'topic':tt['label'],
                'score_list':prob_dict
            }
            porn.append(content)

2805959it [41:01, 1139.99it/s]


In [16]:
with open('/data/albert.xht/xiaoda/sentiment/green_porn/short_text_benchmark_for_cro_albert_seqing.json.txt.offensive', 'w') as fwobj:
    for d in tqdm(porn):
        fwobj.write(json.dumps(d, ensure_ascii=False)+'\n')

100%|██████████| 2805958/2805958 [00:43<00:00, 63985.46it/s]


In [13]:
with open('/data/albert.xht/xiaoda/sentiment/green_politics/short_text_benchmark_for_cro_albert_politics.json.txt.offensive', 'w') as fwobj:
    for d in tqdm(offensive):
        fwobj.write(json.dumps(d, ensure_ascii=False)+'\n')

100%|██████████| 456219/456219 [00:08<00:00, 55401.56it/s]


In [32]:

mapping = {
    'black':'涉政',
    'white':'正常'
}

import random
random.shuffle(offensive)

with open('/data/albert.xht/xiaoda/sentiment/green_politics/green_politics.json', 'w') as fwobj:
    black = []
    left = []
    count1 = 0
    data_dict = {}
    for d in tqdm(offensive):
        p = {
                'text':d['text'],
                'label':[mapping[d['topic']]]
            }
        if d['text'] not in data_dict:
            data_dict[d['text']] = set()
        data_dict[d['text']].add(p['label'][0])
    for key in tqdm(data_dict):
        label = data_dict[key]
        if len(label) == 1:
            p = {
                'text':key,
                'label':list(label)
            }
        fwobj.write(json.dumps(p, ensure_ascii=False)+'\n')
        
    
    #     if d['score_list']['offensive'][0][1] > 0.9 and d['score_list']['query_risk'][0][1] > 0.9:
    #         p = {
    #             'text':d['text'],
    #             'label':[mapping[d['topic']]],
    #             'source':'risk-offensive'
    #         }
    #         fwobj.write(json.dumps(p, ensure_ascii=False)+'\n')
    #         count1 += 1
    #         if count1 >= 50000:
    #             break
    # count2 = 0
    # for d in offensive:
    #     if d['topic'] not in mapping:
    #         continue
    #     if d['score_list']['offensive'][0][1] < 0.1 and d['score_list']['query_risk'][0][1] < 0.1:
    #         p = {
    #             'text':d['text'],
    #             'label':[mapping[d['topic']]],
    #             'source':'politics'
    #         }
    #         fwobj.write(json.dumps(p, ensure_ascii=False)+'\n')
    #         count2 += 1
    #         if count2 >= 50000:
    #             break
print(count1, count2)

100%|██████████| 451726/451726 [00:03<00:00, 131818.44it/s]


0 0


In [17]:
porn = []
with open('/data/albert.xht/xiaoda/sentiment/green_porn/short_text_benchmark_for_cro_albert_seqing.json.txt.offensive') as frobj:
    for line in frobj:
        porn.append(json.loads(line.strip()))

In [30]:

mapping = {
    '色情':'色情',
    '色情-白样本':'正常',
    '色情-正常':'正常'
}

import random
random.shuffle(porn)

data_dict = {}

with open('/data/albert.xht/xiaoda/sentiment/green_porn/green_porn.json', 'w') as fwobj:
    black = []
    left = []
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    for d in tqdm(porn):
        p = {
                'text':d['text'],
                'label':[mapping[d['topic']]]
            }
        if d['text'] not in data_dict:
            data_dict[d['text']] = set()
        data_dict[d['text']].add(p['label'][0])
    for key in tqdm(data_dict):
        label = data_dict[key]
        if len(label) == 1:
            p = {
                'text':key,
                'label':list(label)
            }
        fwobj.write(json.dumps(p, ensure_ascii=False)+'\n')


100%|██████████| 2758518/2758518 [00:19<00:00, 140611.64it/s]


(0, 0, 0, 0)

In [25]:
low = []
for d in porn:
    if d['score_list']['offensive'][0][1] < 0.5 and d['score_list']['query_risk'][0][1] < 0.5 and d['score_list']['offensive'][0][1] > 0.4:
        if d['topic'] in ['色情-白样本']:
            low.append(d)

In [26]:
low[0]

{'text': '刹住个屁一个村一年都能吃好几万他这一个县呢',
 'topic': '色情-白样本',
 'score_list': {'offensive': [['冒犯', 0.45606398582458496],
   ['正常', 0.5439360737800598]],
  'query_risk': [['风险', 0.4721612334251404],
   ['个人信息', 1.2448796042008325e-05],
   ['正常', 0.5278263688087463]]}}

In [33]:
tttt = []
for d in offensive:
    if d['score_list']['query_risk'][0][1] >= 0.8 and d['topic'][0] in ['军事', '时事政治', '历史', '国家']:
        tttt.append(d)

In [67]:
positive = []
neg = []
with open('/data/albert.xht/xiaodao/topic_classification_v7/biake_qa_web_text_zh_train.json.positive.topic.v10') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['topic'][0][0] in ['军事', '时事政治', '历史', '国家']:
            if content['score_list']['intent'][0][1] + content['score_list']['intent'][1][1] > 0.4:
                neg.append(content)
            else:
                positive.append(content)
        else:
            positive.append(content)
            
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/embed_linear_small_white.json.topic.v10') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['topic'][0][0] in ['军事', '时事政治', '历史', '国家']:
            if content['score_list']['intent'][0][1] + content['score_list']['intent'][1][1] > 0.4:
                neg.append(content)
            else:
                positive.append(content)
        else:
            positive.append(content)


In [46]:

input_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/smal_white_positive.json.filter'
output_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/smal_white_positive.json.filter.offensive_query_risk'


batch_inference(input_path, output_path)


128it [00:00, 1257.45it/s]

/data/albert.xht/raw_chat_corpus/topic_classification_v4/smal_white_positive.json.filter ===input-path===
/data/albert.xht/raw_chat_corpus/topic_classification_v4/smal_white_positive.json.filter.offensive_query_risk ===output-path===


44674it [00:19, 2305.49it/s]


In [54]:
black, white = [], []
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/smal_white_positive.json.filter.offensive_query_risk') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['score_list']['query_risk'][0][1] > 0.3 and content['score_list']['offensive'][0][1] < 0.5:
            black.append(content)
        elif content['score_list']['query_risk'][0][1] < 0.3 and content['score_list']['offensive'][0][1] < 0.3:
            white.append(content)

In [60]:
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/smal_white_positive.json.filter.offensive_query_risk', 'w') as fwobj:
    for d in white:
        content = {
            'text':d['text'],
            'label':['正常']
        }
        fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')

In [70]:
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/smal_white_positive.json.filter', 'w') as fwobj:
    for d in positive:
        content = {
            'text':d['text'],
            'label':['正常']
        }
        fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
        

In [11]:

offensive = []
offensive_dict = {}
with open('/data/albert.xht/xiaodao/query_risk_v10/biake_qa_web_text_zh_train.json.offensive.all.v10.1') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['score_list']['offensive'][0][1] < 0.8 and content['score_list']['offensive'][0][1] >= 0.5:
            offensive.append(content)
            if content['text'] not in offensive_dict:
                offensive_dict[content['text']] = []
            offensive_dict[content['text']].append('风险')
print(len(offensive_dict), len(offensive))


3176 3177


In [83]:
itag_dict = {}
with open('/data/albert.xht/xiaodao/query_risk_v11/offensive_select_labeled.txt') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['text'] in offensive_dict:
            offensive_dict[content['text']].append(content['label'][0])
        else:
            itag_dict[content['text']] = [content['label'][0]]

In [85]:
oooo = []
for text in offensive_dict:
    if len(set(offensive_dict[text])) > 1:
        oooo.append(text)

In [88]:
with open('/data/albert.xht/xiaodao/query_risk_v10/biake_qa_web_text_zh_train.json.offensive.all.v10.1.offensive', 'w') as fwobj:
    for d in offensive:
        content = {
            'text':d['text'],
            'label':['风险']
        }
        fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
        


In [21]:
topic = []
topic_other = []
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.all_risk.v9.1') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['topic'][0] in ['军事', '时事政治', '历史', '国家']:
            if content['score_list']['intent'][0][1] + content['score_list']['intent'][1][1] > 0.4:
                topic.append(content)
            else:
                topic_other.append(content)



In [148]:
def batch_inference(input_path, output_path):
    from tqdm import tqdm
    import numpy as np
    import json, re

    def risk_predict_batch(text):
        if isinstance(text, list):
            text_list = text
        else:
            text_list = [text]
        result_list = risk_api.predict_batch(text_list)
        return result_list
    
    print(input_path, '===input-path===')
    print(output_path, '===output-path===')
    
    with open(output_path, 'w') as fwobj:
        with open(input_path, 'r') as frobj:
            queue = []
            t = []
            for line in tqdm(frobj):
                content = json.loads(line.strip())
                content['text'] = re.sub('请问', '', content['text'])
                text = re.sub(r"([，\_《。》、？；：‘’＂“”【「】」·！@￥…（）—\,\<\.\>\/\?\;\:\'\"\[\]\{\}\~\`\!\@\#\$\%\^\&\*\(\)\-\=\+])+", "", content['text'])   # 合并正文中过多的空格
                if len(text) >= 192:
                    continue
                queue.append(text)
                t.append(content)
                if np.mod(len(queue), 128) == 0:
                    probs = risk_predict_batch(queue)
                    for prob_dict, text, tt in zip(probs, queue, t):
                        # content = {
                        #     'text':tt['text'],
                        #     'topic':tt['topic'],
                        #     'score_list':prob_dict,
                        #     'label'
                        #     # 'score_list': tt['score_list']
                        # }
                        tt['score_list'] = prob_dict
                        fwobj.write(json.dumps(tt, ensure_ascii=False)+'\n')
                    queue = []
                    t = []
            if queue:
                probs = risk_predict_batch(queue)
                for prob_dict, text, tt in zip(probs, queue, t):
                    # content = {
                    #     'text':tt['text'],
                    #     'topic':tt['topic'],
                    #     'score_list':prob_dict,
                    #     # 'score_list': tt['score_list']
                    # }
                    tt['score_list'] = prob_dict
                    fwobj.write(json.dumps(tt, ensure_ascii=False)+'\n')
                    



In [43]:
input_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.topic.knn.final'
output_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.offensive_query_risk'


batch_inference(input_path, output_path)

256it [00:00, 1793.53it/s]

/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.topic.knn.final ===input-path===
/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.offensive_query_risk ===output-path===


2071401it [14:07, 2444.75it/s]


In [ ]:
input_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.topic.knn.final'
output_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.offensive_query_risk'


batch_inference(input_path, output_path)

In [128]:
black, white = [], []
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.offensive_query_risk') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['score_list']['offensive'][0][1] < 0.3 and content['score_list']['query_risk'][0][1] >= 0.7:
            black.append(content)
        elif content['score_list']['offensive'][0][1] > 0.7 and content['score_list']['query_risk'][0][1] <= 0.3:
            white.append(content)
        

8125

In [132]:
text_dict = {}
with open('/data/albert.xht/xiaodao/query_risk_v11/offensive_select_labeled.txt.topic') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['label'][0] in ['风险']:
            if content['text'] not in text_dict:
                text_dict[content['text']] = []
            text_dict[content['text']].append(content)

with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.offensive_query_risk.itag', 'w') as fwobj:
    fwobj.write("&&&".join(['query'])+'\n')
    for d in black:
        if d['text'] not in text_dict:
            fwobj.write("&&&".join([d['text']])+'\n')
    for key in text_dict:
        fwobj.write("&&&".join([key])+'\n')
        
    with open('/data/albert.xht/sentiment/offensive_cold.json.offensive_query_risk') as frobj:
        for line in frobj:
            content = json.loads(line.strip())
            if content['score_list']['query_risk'][0][1] > 0.8 and content['topic'][0] in ['正常']:
                fwobj.write("&&&".join([content['text']])+'\n')
                
    with open('/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other.offensive_query_risk') as frobj:
        for line in frobj:
            content = json.loads(line.strip())
            content['text'] = re.sub('[男女]+', ' ', content['text'])
            if content['score_list']['query_risk'][0][1] > 0.8 and content['score_list']['offensive'][0][1] < 0.3:
                fwobj.write("&&&".join([content['text']])+'\n')
                
    for d in sucide:
        d['title'] = re.sub('[男女]+', ' ', d['title'])
        fwobj.write("&&&".join([d['title']])+'\n')
        
    for d in white:
        fwobj.write("&&&".join([d['text']])+'\n')

   
        

In [150]:
risk_api.predict('垃圾的社会主义征程')

{'offensive': [['冒犯', 0.9479578733444214], ['正常', 0.05204217880964279]],
 'query_risk': [['风险', 0.8904106020927429],
  ['个人信息', 4.8662836888979655e-06],
  ['正常', 0.10958455502986908]]}

In [98]:
ss = []
for d in white:
    if d['topic'][0] in ['军事']:
        ss.append(d)

In [102]:
s = []
for d in white:
    if d['topic'][0] in ['时事政治']:
        s.append(d)

In [105]:
len(s)

5652

In [81]:
input_path = '/data/albert.xht/sentiment/offensive_cold.json'
output_path = '/data/albert.xht/sentiment/offensive_cold.json.offensive_query_risk'


batch_inference(input_path, output_path)




128it [00:00, 755.41it/s]

/data/albert.xht/sentiment/offensive_cold.json ===input-path===
/data/albert.xht/sentiment/offensive_cold.json.offensive_query_risk ===output-path===


25663it [00:21, 1198.03it/s]


In [43]:
black, white = [], []
with open('/data/albert.xht/sentiment/offensive_cold.json.offensive_query_risk') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['score_list']['query_risk'][0][1] > 0.8 and content['score_list']['offensive'][0][1] < 0.5:
            black.append(content)
        elif content['score_list']['query_risk'][0][1] < 0.3 and content['score_list']['offensive'][0][1] < 0.3:
            white.append(content)

In [55]:
black = []
with open(output_path) as frobj:
    for line in tqdm(frobj):
        content = json.loads(line.strip())
        if content['score_list']['query_risk'][0][1] > 0.3 and content['score_list']['query_risk'][0][1] < 0.5:
            black.append(content)

2071401it [00:19, 104403.54it/s]


In [7]:
input_path = '/data/albert.xht/sentiment/green_teenager.json.all.detail'
output_path = '/data/albert.xht/sentiment/green_teenager.json.all.detail.offensive_query_risk'


batch_inference(input_path, output_path)

0it [00:00, ?it/s]

/data/albert.xht/sentiment/green_teenager.json.all.detail ===input-path===
/data/albert.xht/sentiment/green_teenager.json.all.detail.offensive_query_risk ===output-path===


241035it [03:22, 1188.86it/s]


In [ ]:

input_path = '/data/albert.xht/sentiment/green_teenager.json.all.detail'
output_path = '/data/albert.xht/sentiment/green_teenager.json.all.detail.offensive_query_risk'


batch_inference(input_path, output_path)



In [81]:
with open('/data/albert.xht/sentiment/green_teenager.json.all.detail.offensive_query_risk.filter', 'w') as fwobj:
    with open('/data/albert.xht/sentiment/green_teenager.json.all.detail.offensive_query_risk') as frobj:
        white = []
        black = []
        for line in frobj:
            content = json.loads(line.strip())
            # content['text'] = re.sub('[男女]+', ' ', content['text'])
            if content['score_list']['query_risk'][0][1]< 0.2 and content['score_list']['offensive'][0][1] < 0.3:
                white.append(content)
            elif content['score_list']['query_risk'][0][1]> 0.9 and content['score_list']['offensive'][0][1] > 0.9:
                black.append(content)
    for d in white:
        tmp = {
            'text':d['text'],
            'label':['正常'],
            'source':'green_teenager_filter'
        }
        fwobj.write(json.dumps(tmp, ensure_ascii=False)+'\n')
    for d in black:
        tmp = {
            'text':d['text'],
            'label':['风险'],
            'source':'green_teenager_filter'
        }
        fwobj.write(json.dumps(tmp, ensure_ascii=False)+'\n')
    
            

In [120]:
s1_mapping = {
    '1.1':'学业烦恼、对未来规划的迷茫',
    '1.2':'事业和工作烦恼',
    '1.3':'家庭问题和矛盾',
    '1.4':'物质滥用',
    '1.5':'悲恸',
    '1.6':'失眠',
    '1.7':'压力',
    '1.8':'人际关系',
    '1.9':'情感关系问题',
    '1.10':'离婚',
    '1.11':'分手',
    '1.12':'自我探索',
    '1.13':'低自尊',
    '1.14':'青春期问题',
    '1.15':'强迫症',
    '1.16':'其它',
    '1.17':'男同性恋、女同性恋、双性恋与跨性别',
    '1.18':'性问题',
    '1.19':'亲子关系'
}

s2_mapping = {
    '2.1':'忧郁症',
    '2.2':'焦虑症',
    '2.3':'躁郁症',
    '2.4':'创伤后应激反应',
    '2.5':'恐慌症',
    '2.6':'厌食症和暴食症',
    '2.7':'非疾病',
    '2.8':'其它疾病'
}

s3_mapping = {
    '3.1':'正在进行的自杀行为',
    '3.2':'策划进行的自杀行为',
    '3.3':'自残',
    '3.4':'进行的人身伤害',
    '3.5':'计划的人身伤害',
    '3.6':'无伤害身体倾向'
}

sucide = []

with open("/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other", "w") as fwobj:
    with open("/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8", "r") as frobj:
        for line in frobj:
            content = json.loads(line.strip())
            title = re.sub('[男女]+', ' ', content['title']) #''.join(re.split('[\s,]', content['title'])[1:])
            content['title'] = title
            if len(title) >= 5:
                if  s3_mapping[content['label']['s3']] in ['正在进行的自杀行为', '策划进行的自杀行为', '自残']:
                    sucide.append(content)
                    continue
                else:
                    tmp = {
                        'text':title,
                        'label':[s3_mapping[content['label']['s3']],
                                s1_mapping[content['label']['s1']],
                                s2_mapping[content['label']['s2']]
                                ],
                        'source':'efaqa'
                    }
                    fwobj.write(json.dumps(tmp, ensure_ascii=False)+'\n')

In [73]:
input_path = '/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other'
output_path = '/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other.offensive_query_risk'


batch_inference(input_path, output_path)

256it [00:00, 2285.15it/s]

/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other ===input-path===
/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other.offensive_query_risk ===output-path===


19655it [00:10, 1874.85it/s]


In [198]:
risk_api.predict('最后一集ymd出来耍帅耍了个够…不过我真是佩服小变态脸都毁了还能借到三百万[拜拜][拜拜][拜拜]我以为要教我看清这看脸的世界…不过这季居然有两人上岸…不过大家都还背着债啊')

{'offensive': [['冒犯', 0.1621621698141098], ['正常', 0.837837815284729]],
 'query_risk': [['风险', 0.8829036355018616],
  ['个人信息', 9.993584535550326e-05],
  ['正常', 0.11699634790420532]]}

In [65]:
s= """{"chats": [{"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:01:41", "type": "textMessage", "value": "公交车要承担赔偿责任"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:02:17", "type": "textMessage", "value": "双方均有责任"}, {"label": {"attitude": "", "s3": ""}, "sender": "owner", "time": "16:02:54", "type": "textMessage", "value": "大概能陪多少！算合适！"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:03:41", "type": "textMessage", "value": "具体要由交警依据相关证据做出责任认定"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:04:19", "type": "textMessage", "value": "具体数额要看伤情。这里涉及保险赔付问题"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:04:20", "type": "textMessage", "value": "然后根据损失情况确定赔偿金额"}, {"label": {"attitude": "", "s3": ""}, "sender": "owner", "time": "16:06:43", "type": "textMessage", "value": "昏迷三天头部缝三针   交警判定责任是     电动车车主百分之六十  公交车百分之四十   其他没什么了  怎么赔偿？"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:07:36", "type": "textMessage", "value": "伤情，对方是否做了伤情鉴定"}, {"label": {"attitude": "", "s3": ""}, "sender": "owner", "time": "16:08:03", "type": "textMessage", "value": "轻伤"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:08:08", "type": "textMessage", "value": "须在治疗终结伤残鉴定后才能确定"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:08:40", "type": "textMessage", "value": "这里有保险，详情，或者具体操作事宜建议电话咨询。"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:08:41", "type": "textMessage", "value": "具体可以电话咨询或关注私聊"}, {"label": {"attitude": "", "s3": ""}, "sender": "owner", "time": "16:08:52", "type": "textMessage", "value": "鉴定结果是轻伤   怎么赔偿"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:10:21", "type": "textMessage", "value": "伤残鉴定"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:13:16", "type": "textMessage", "value": "你的轻伤是伤情鉴定"}, {"label": {"knowledge": false, "negative": false, "question": false}, "sender": "audience", "time": "16:13:32", "type": "textMessage", "value": "赔偿需要伤残鉴定"}, {"label": {"attitude": "", "s3": ""}, "sender": "owner", "time": "16:13:40", "type": "textMessage", "value": "十级"}, {"label": {"attitude": "", "s3": ""}, "sender": "owner", "time": "16:14:15", "type": "textMessage", "value": "基>本上没什么事！"}, {"label": {"attitude": "", "s3": ""}, "sender": "owner", "time": "16:14:18", "type": "textMessage", "value": "基本上没什么事！"}], "crawldate": "2020-03-02 18:10:27.338000", "date": "2015-06-26 16:00:02", "label": {"s1": "1.16", "s2": "2.7", "s3": "3.4"}, "owner": "匿名", "title": "你好！闯红灯的公交车和闯红灯的电动车相撞，电动车车主被撞昏迷头部出血  责任怎么处理。"}
"""

In [68]:
content = json.loads(s)

In [69]:
title = ''.join(re.split('[\s,]', content['title'])[1:])

In [186]:
risk_api.predict('中共一大都干了什么')

{'offensive': [['冒犯', 0.5700234174728394], ['正常', 0.42997655272483826]],
 'query_risk': [['风险', 0.9904796481132507],
  ['个人信息', 1.3330283763934858e-05],
  ['正常', 0.009506949223577976]]}

In [78]:
import re
with open('/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.topic.offensive_query_risk') as frobj:
    black = []
    for line in frobj:
        content = json.loads(line.strip())
        content['text'] = re.sub('[男女]+', ' ', content['text'])
        if content['score_list']['query_risk'][0][1]< 0.3 and content['score_list']['offensive'][0][1] < 0.3:
            black.append(content)
        

In [166]:
risk_api.predict('王一博的电话号码')

{'offensive': [['冒犯', 0.0024976327549666166], ['正常', 0.9975023865699768]],
 'query_risk': [['风险', 4.455989983398467e-05],
  ['个人信息', 0.9625425934791565],
  ['正常', 0.03741287812590599]]}

In [79]:
data_dict = {}
import re
with open('/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other.offensive_query_risk') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        content['text'] = re.sub('[男女]+', ' ', content['text'])
        if content['score_list']['query_risk'][0][1]< 0.3 and content['score_list']['offensive'][0][1] < 0.3:
            if content['text'] not in data_dict:
                data_dict[content['text']] = []
            data_dict[content['text']].append(content)

with open('/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8.other.offensive_query_risk.white', 'w') as fwobj:
    for d in black:
        if d['topic'][0][0] in ['法律']:
            continue
        if d['text'] not in data_dict:
            continue
        tmp = {
            'text':d['text'],
            'label':['正常'],
            'source':'efaqa'
        }
        fwobj.write(json.dumps(tmp, ensure_ascii=False)+'\n')

In [121]:
s1_mapping = {
    '1.1':'学业烦恼、对未来规划的迷茫',
    '1.2':'事业和工作烦恼',
    '1.3':'家庭问题和矛盾',
    '1.4':'物质滥用',
    '1.5':'悲恸',
    '1.6':'失眠',
    '1.7':'压力',
    '1.8':'人际关系',
    '1.9':'情感关系问题',
    '1.10':'离婚',
    '1.11':'分手',
    '1.12':'自我探索',
    '1.13':'低自尊',
    '1.14':'青春期问题',
    '1.15':'强迫症',
    '1.16':'其它',
    '1.17':'男同性恋、女同性恋、双性恋与跨性别',
    '1.18':'性问题',
    '1.19':'亲子关系'
}

s2_mapping = {
    '2.1':'忧郁症',
    '2.2':'焦虑症',
    '2.3':'躁郁症',
    '2.4':'创伤后应激反应',
    '2.5':'恐慌症',
    '2.6':'厌食症和暴食症',
    '2.7':'非疾病',
    '2.8':'其它疾病'
}

s3_mapping = {
    '3.1':'正在进行的自杀行为',
    '3.2':'策划进行的自杀行为',
    '3.3':'自残',
    '3.4':'进行的人身伤害',
    '3.5':'计划的人身伤害',
    '3.6':'无伤害身体倾向'
}

efaqa = []
with open("/data/albert.xht/pretrained_model_risk/corpus/efaqa-corpus-zh/efaqa-corpus-zh.utf8", "r") as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        title = content['title'] #''.join(re.split('[\s,]', content['title'])[1:])
        if len(title) >= 5:
            if  s3_mapping[content['label']['s3']] in ['正在进行的自杀行为', '策划进行的自杀行为', '自残']:
                tmp = {
                    'text':title,
                    'label':['风险'],
                    'source':'efaqa'
                }
                continue
            elif s3_mapping[content['label']['s3']] in ['无伤害身体倾向', '计划的人身伤害'] and \
            s1_mapping[content['label']['s1']] not in ['其他', '男同性恋、女同性恋、双性恋与跨性别']:
                tmp = {
                    'text':title,
                    'label':['正常'],
                    'source':'efaqa'
                }
                efaqa.append(content)

In [109]:
with open('/data/albert.xht/sentiment/offensive_cold.json.offensive_query_risk') as frobj:
    black = []
    for line in frobj:
        content = json.loads(line.strip())
        if content['score_list']['query_risk'][0][1] < 0.3 and content['score_list']['offensive'][0][1] < 0.3 and content['topic'][0] not in ['冒犯']:
            black.append(content)

In [151]:
risk_api.predict('和不同的女人做爱感觉差异真的很大吗')

{'offensive': [['冒犯', 0.21428874135017395], ['正常', 0.7857112288475037]],
 'query_risk': [['风险', 0.6982224583625793],
  ['个人信息', 6.725983439537231e-06],
  ['正常', 0.30177077651023865]]}